In [1]:

%pprint

Pretty printing has been turned OFF



## Let's use our labeled data to evaluate a Conditional Random Fields model

In [2]:

%%time
%run ../py/html_analysis.py

crf = CrfUtilities()

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\CRF.pickle
Wall time: 10.8 s


In [12]:

files_list = crf.su.get_files_list(crf.su.cursor, verbose=False)
crf.CRF.predict_single(xseq=feature_dict_list)

C:\Users\dev\Documents\Repositories\job-hunting\jh\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Signature: crf.CRF.predict_single(xseq)
Source:   
    def predict_single(self, xseq):
        """
        Make a prediction.

        Parameters
        ----------
        xseq : list of dicts
            feature dicts in python-crfsuite format

        Returns
        -------
        y : list of strings
            predicted labels

        """
        return self.tagger_.tag(xseq)
File:      c:\users\dev\documents\repositories\job-hunting\jh\lib\site-packages\sklearn_crfsuite\estimator.py
Type:      method


In [6]:

[f'crf.CRF.{fn}' for fn in dir(crf.CRF) if 'predict' in fn.lower()]

['crf.CRF.predict', 'crf.CRF.predict_marginals', 'crf.CRF.predict_marginals_single', 'crf.CRF.predict_single']

In [11]:


import sklearn_crfsuite

CRF = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100,
                           all_possible_transitions=True)
HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
X_train = []
y_train = []
HEADER_PATTERN_DICT

{'AVP,_Data_Science_-_Fort_Mill,_SC_29715_-_Indeed.com_bba3b2578c9adb92.html': [{'initial_tag': 'h2', 'is_header': True, 'is_task_scope': True, 'is_minimum_qualification': False, 'is_preferred_qualification': False, 'is_legal_notification': False, 'is_job_title': False, 'is_office_location': False, 'is_job_duration': False, 'is_supplemental_pay': False, 'is_educational_requirement': False, 'is_interview_procedure': False, 'is_corporate_scope': False, 'is_posting_date': False, 'is_other': False, 'child_str': '<h2 class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</h2>'}, {'initial_tag': 'p', 'is_header': False, 'child_str': '<p>Do you a passion for applying analytics tools to uncover novel insights that drive real business outcomes? Would you like to be part of a small, high-functioning team, with room to grow your career? Do you already have 2-4 years of experience applying machine learning to business problems, and

In [9]:

crf.CRF.predict_marginals_single(sent_str)

[{'H-TS': 0.035254343736521615, 'O': 0.1071879749050959, 'H-PQ': 0.03329944997600356, 'H-RQ': 0.03438742620665897, 'H-CS': 0.03844229784460139, 'O-SP': 0.042172798566390814, 'O-TS': 0.03862452872444592, 'O-LN': 0.04296079495460932, 'H-LN': 0.0345664267677794, 'H-JT': 0.036527895926544665, 'H-OL': 0.041454313003874546, 'H-JD': 0.03549722966392512, 'H-SP': 0.037587156794800784, 'O-OL': 0.0356444976293885, 'H-ER': 0.035334481015611464, 'H-IP': 0.03500886730704661, 'O-RQ': 0.04189850115417665, 'O-CS': 0.04217475461234067, 'O-PQ': 0.03628360853640577, 'H-PD': 0.03290031318214735, 'O-ER': 0.034561959580296404, 'H-O': 0.03239699304814214, 'H': 0.043964043770953205, 'O-O': 0.03284527250368107, 'O-IP': 0.03902407058855792}, {'H-TS': 0.027623220763210495, 'O': 0.244393936696699, 'H-PQ': 0.026402727370210894, 'H-RQ': 0.028288842614579517, 'H-CS': 0.029981795661265406, 'O-SP': 0.03960708634444898, 'O-TS': 0.033689064340936214, 'O-LN': 0.03669024826343864, 'H-LN': 0.028087206960488185, 'H-JT': 0.02

In [ ]:

sent_str = 'You work here.'
crf.CRF.predict_single(sent_str)

In [ ]:

[f'lr.{fn}' for fn in dir(lr) if not fn.startswith('_')]

In [ ]:

import random

print('\nCV')
print(lr.CV)
print('\nLR_DICT')
for k, v in random.sample(lr.LR_DICT.items(), 2):
    print(k, v)
print('\nPREDICT_PERCENT_FIT_DICT')
for k, v in random.sample(lr.PREDICT_PERCENT_FIT_DICT.items(), 2):
    print(k, v)
print('\nTT')
print(lr.TT)


## Test its accuracy

In [ ]:

sql_str = f'''
    SELECT
        np.[navigable_parent],
        pos.[pos_symbol]
    FROM
        [Jobhunting].[dbo].[NavigableParents] np INNER JOIN
        [Jobhunting].[dbo].[PartsOfSpeech] pos ON
            pos.[is_header] = np.[is_header] AND
            pos.[is_task_scope] = np.[is_task_scope] AND
            pos.[is_minimum_qualification] = np.[is_minimum_qualification] AND
            pos.[is_preferred_qualification] = np.[is_preferred_qualification] AND
            pos.[is_legal_notification] = np.[is_legal_notification] AND
            pos.[is_job_title] = np.[is_job_title] AND
            pos.[is_office_location] = np.[is_office_location] AND
            pos.[is_job_duration] = np.[is_job_duration] AND
            pos.[is_supplemental_pay] = np.[is_supplemental_pay] AND
            pos.[is_educational_requirement] = np.[is_educational_requirement] AND
            pos.[is_interview_procedure] = np.[is_interview_procedure] AND
            pos.[is_corporate_scope] = np.[is_corporate_scope] AND
            pos.[is_posting_date] = np.[is_posting_date] AND
            pos.[is_other] = np.[is_other];'''
pos_df = pd.DataFrame(lr.su.get_execution_results(lr.cursor, sql_str, verbose=False))
POS_DICT = pos_df.set_index('navigable_parent')['pos_symbol'].to_dict()

In [ ]:

y_true = []
y_pred = []
for sent_str, pos_symbol in POS_DICT.items():
    y_true.append(pos_symbol)
    y_pred.append(lr.lr_predict_single(sent_str))

In [ ]:

from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

In [ ]:

seen = set()
seen_add = seen.add
label_list = [x for x in y_true if not (x in seen or seen_add(x))]

In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred, labels=label_list)
fig, ax = plt.subplots(figsize=(18, 6))
cm_display = ConfusionMatrixDisplay(cm).plot(ax=ax)
xticklabels_list = ax.set_xticklabels(label_list, rotation='vertical')
yticklabels_list = ax.set_yticklabels(label_list)